<a href="https://colab.research.google.com/github/ezevischi/capstone/blob/master/Capstone_Project_The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working in the office regularly, I travel through different nearby neighborhoods with my car, therefore I generally need gasoline at a service station. Foursquare will let me know the gas stations near the office. In turn, it would need to load gasoline at "Shell" brand service stations. In this way, obtaining the geo coordinates of the office, you could obtain the previously detailed information.

In [88]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Folium installed
Libraries imported.


In [71]:
CLIENT_ID = 'MBQBAT4QEUESXYG2QMD1BPHRFNTPJA1QPRXWDE5SIZ30IOM2' # your Foursquare ID
CLIENT_SECRET = 'YNQKOKFRAVV3NLMRW3C0ID43TQ2J0CBJEFAWSH1S2J04VRJI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MBQBAT4QEUESXYG2QMD1BPHRFNTPJA1QPRXWDE5SIZ30IOM2
CLIENT_SECRET:YNQKOKFRAVV3NLMRW3C0ID43TQ2J0CBJEFAWSH1S2J04VRJI


In [123]:
#address of my office
address = 'Av. del Libertador 7202, Buenos Aires'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-34.5467627 -58.4576083


In [124]:
#I want to know the fuel stations near to my office
search_query = 'Shell'
radius = 6000
print(search_query + ' .... OK!')

Shell .... OK!


In [125]:
#I look at in Foursquare
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MBQBAT4QEUESXYG2QMD1BPHRFNTPJA1QPRXWDE5SIZ30IOM2&client_secret=YNQKOKFRAVV3NLMRW3C0ID43TQ2J0CBJEFAWSH1S2J04VRJI&ll=-34.5467627,-58.4576083&v=20180604&query=Shell&radius=6000&limit=40'

In [126]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f443be1ec297935bfa5cf05'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gas_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d113951735',
      'name': 'Gas Station',
      'pluralName': 'Gas Stations',
      'primary': True,
      'shortName': 'Gas Station'}],
    'hasPerk': False,
    'id': '4c7321eed944224b0dbb0e29',
    'location': {'address': 'Av. Figueroa Alcorta 7600',
     'cc': 'AR',
     'city': 'Buenos Aires',
     'country': 'Argentina',
     'crossStreet': 'esquina Udaondo',
     'distance': 529,
     'formattedAddress': ['Av. Figueroa Alcorta 7600 (esquina Udaondo)',
      'Buenos Aires',
      'Buenos Aires C.F.',
      'Argentina'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -34.54653639219639,
       'lng': -58.451841698590336}],
     'lat': -34.54653639219639,
     'lng': -58.451841698590336,
     'state': 'Buenos Aires C.F.'},
    'name': 'Shell'

In [127]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,venuePage.id
0,4c7321eed944224b0dbb0e29,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598307564,False,Av. Figueroa Alcorta 7600,esquina Udaondo,-34.546536,-58.451842,"[{'label': 'display', 'lat': -34.5465363921963...",529,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Av. Figueroa Alcorta 7600 (esquina Udaondo), ...",NaN,NaN
1,4b86f7b6f964a520a5a831e3,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598307564,False,Av. de los Incas 3019,esq. Av. Crámer,-34.568959,-58.457548,"[{'label': 'display', 'lat': -34.5689594210926...",2470,AR,Belgrano,Buenos Aires C.F.,Argentina,"[Av. de los Incas 3019 (esq. Av. Crámer), C14...",C1426ELA,NaN
2,4bfc2a47d6f2c9b610ce4ec8,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598307564,False,Av. del Libertador 7920,esq. Av. Comodoro Martín Rivadavia,-34.541944,-58.463151,"[{'label': 'display', 'lat': -34.5419444839943...",738,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. del Libertador 7920 (esq. Av. Comodoro Ma...,C1429BMZ,NaN
3,5f35708c83a8781d8ca91e6f,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598307564,False,Avenida Presidente Figueroa Alcorta 7786,NaN,-34.546543,-58.451910,"[{'label': 'display', 'lat': -34.5465429, 'lng...",523,AR,Capital Federal,Buenos Aires,Argentina,"[Avenida Presidente Figueroa Alcorta 7786, 142...",1429,594433018
4,4ce09ec870bba1cd7d846dc4,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598307564,False,Av. Gral. Paz 3802,e/ Av. de los Constituyentes y Av. Balbín,-34.559611,-58.504907,"[{'label': 'display', 'lat': -34.5596109358933...",4566,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. Gral. Paz 3802 (e/ Av. de los Constituyen...,1419,NaN


In [128]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,id
0,Shell,Gas Station,Av. Figueroa Alcorta 7600,esquina Udaondo,-34.546536,-58.451842,"[{'label': 'display', 'lat': -34.5465363921963...",529,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Av. Figueroa Alcorta 7600 (esquina Udaondo), ...",NaN,4c7321eed944224b0dbb0e29
1,Shell,Gas Station,Av. de los Incas 3019,esq. Av. Crámer,-34.568959,-58.457548,"[{'label': 'display', 'lat': -34.5689594210926...",2470,AR,Belgrano,Buenos Aires C.F.,Argentina,"[Av. de los Incas 3019 (esq. Av. Crámer), C14...",C1426ELA,4b86f7b6f964a520a5a831e3
2,Shell,Gas Station,Av. del Libertador 7920,esq. Av. Comodoro Martín Rivadavia,-34.541944,-58.463151,"[{'label': 'display', 'lat': -34.5419444839943...",738,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. del Libertador 7920 (esq. Av. Comodoro Ma...,C1429BMZ,4bfc2a47d6f2c9b610ce4ec8
3,Shell,Gas Station,Avenida Presidente Figueroa Alcorta 7786,NaN,-34.546543,-58.451910,"[{'label': 'display', 'lat': -34.5465429, 'lng...",523,AR,Capital Federal,Buenos Aires,Argentina,"[Avenida Presidente Figueroa Alcorta 7786, 142...",1429,5f35708c83a8781d8ca91e6f
4,Shell,Gas Station,Av. Gral. Paz 3802,e/ Av. de los Constituyentes y Av. Balbín,-34.559611,-58.504907,"[{'label': 'display', 'lat': -34.5596109358933...",4566,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. Gral. Paz 3802 (e/ Av. de los Constituyen...,1419,4ce09ec870bba1cd7d846dc4
5,Shell,Gas Station,AV.DEL LIBERTADOR 7920-90,NaN,-34.541921,-58.463166,"[{'label': 'display', 'lat': -34.541921, 'lng'...",741,AR,CAPITAL FEDERAL,Buenos Aires,Argentina,"[AV.DEL LIBERTADOR 7920-90, 1428 CAPITAL FEDER...",1428,5f419429aee48d48c767649e
6,Shell,Gas Station,Av. Pres. Figueroa Alcorta 3099,esq. Mcal. Ramón Castilla,-34.579645,-58.399836,"[{'label': 'display', 'lat': -34.5796447052410...",6437,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. Pres. Figueroa Alcorta 3099 (esq. Mcal. R...,C1425CKJ,4bbea9f898f495216aabd163
7,Shell,Gas Station,Av. Raúl Scalabrini Ortíz 1406,esq. Gorriti,-34.592020,-58.426914,"[{'label': 'display', 'lat': -34.5920201752601...",5770,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. Raúl Scalabrini Ortíz 1406 (esq. Gorriti)...,C1414DOO,4c54958730f92d7f0451b6ba
8,Shell,Gas Station,Av. Cabildo,Deheza,-34.541360,-58.473790,"[{'label': 'display', 'lat': -34.5413603259072...",1600,AR,Capital Federal,Buenos Aires,Argentina,"[Av. Cabildo (Deheza), 1429 Capital Federal, B...",1429,4bf4aa14e5eba59363fe1f90
9,Shell,Gas Station,Av. Santa Fe 3000,esq. Paraná,-34.497533,-58.497543,"[{'label': 'display', 'lat': -34.4975334462650...",6591,AR,Martínez,Buenos Aires,Argentina,"[Av. Santa Fe 3000 (esq. Paraná), Martinez, Bu...",NaN,4c28b9ff97d00f47634240ea


In [131]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around my office

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map